# Description

This notebook aggregates the raw mechanical loading data files into a single file for easy loading and manipulation.

# Imports

In [1]:
import os

import pandas as pd

from IPython.core.display import display, HTML

# Parameters

In [2]:
raw_dirname = "../data/mechanical_loading_datasets"
raw_basenames = ["500mAh1.xlsx","500mAh2.xlsx","500mAh3.xlsx","1500mAh1.xlsx","1500mAh2.xlsx","1500mAh3.xlsx","2000mAh1.xlsx","2000mAh2.xlsx"]

results_dirname = "../data"
results_basename = "raw_mechanical_loading_data.csv.gz"

# Data Loading

## Helper Functions

In [3]:
def process_load_df(load_df):
    load_df = load_df.copy()
    columns = list()
    for column in load_df.columns:
        columns.append((
            "{} SOC".format(int(100*float(column[0]))),
            column[1].title()
        ))
    load_df.columns = pd.MultiIndex.from_tuples(columns)
    
    df_list = list()
    for first_level in load_df.columns.levels[0]:
        columns = [
            (first_level,tmp_second)
            for tmp_first,tmp_second in load_df.columns
            if first_level == tmp_first
        ]
        tmp_df = load_df[columns]
        tmp_df = tmp_df[tmp_df[(first_level,"Time [Sec]")].notna()]
        tmp_df.set_index((first_level,"Time [Sec]"), inplace=True)
        tmp_df.index.rename("Time [Sec]", inplace=True)
        tmp_df.sort_index(axis="index", inplace=True)
        df_list.append(tmp_df)
    return pd.concat(df_list, axis="columns", join="outer")

In [4]:
def process_voltage_df(voltage_df):
    voltage_df = voltage_df.copy()
    columns = list()
    for column in voltage_df.columns:
        columns.append((
            "{} SOC".format(int(100*float(column[0]))),
            column[1].title()
        ))
    voltage_df.columns = pd.MultiIndex.from_tuples(columns)
    
    df_list = list()
    for first_level in voltage_df.columns.levels[0]:
        columns = [
            (first_level,tmp_second)
            for tmp_first,tmp_second in voltage_df.columns
            if first_level == tmp_first
        ]
        tmp_df = voltage_df[columns]
        tmp_df = tmp_df[tmp_df[(first_level,"Time [Sec]")].notna()]
        tmp_df.set_index((first_level,"Time [Sec]"), inplace=True)
        tmp_df.index.rename("Time [Sec]", inplace=True)
        tmp_df.sort_index(axis="index", inplace=True)
        df_list.append(tmp_df)
    return pd.concat(df_list, axis="columns", join="outer")

In [5]:
def process_temperature_df(temperature_df):
    temperature_df = temperature_df.copy()
    columns = list()
    for column in temperature_df.columns:
        first = "{} SOC".format(int(100*float(column[0])))
        if column[1].startswith("time/s") :
            second = "Time [Sec]"
        elif column[1].startswith("temperature") :
            second = "Temperature [C]"
        else:
            raise ValueError(column[1])
        columns.append((first,second))
    temperature_df.columns = pd.MultiIndex.from_tuples(columns)
    
    df_list = list()
    for first_level in temperature_df.columns.levels[0]:
        columns = [
            (first_level,tmp_second)
            for tmp_first,tmp_second in temperature_df.columns
            if first_level == tmp_first
        ]
        tmp_df = temperature_df[columns]
        tmp_df = tmp_df[tmp_df[(first_level,"Time [Sec]")].notna()]
        tmp_df.set_index((first_level,"Time [Sec]"), inplace=True)
        tmp_df.index.rename("Time [Sec]", inplace=True)
        tmp_df.sort_index(axis="index", inplace=True)
#         display(tmp_df)
#         if not tmp_df.index.is_unique:
#             idx = tmp_df.duplicated(keep=False).values
#             print("HERE", sum(idx))
#             print(type(idx))
#             print(idx.shape)
#             print(idx)
#             with pd.option_context("display.min_rows", 20, "display.max_rows", 20, "display.max_columns", None):
#                 display(
#                     tmp_df[idx]
#                 )
#                 display(
#                     tmp_df.index[idx]
#                 )
        df_list.append(tmp_df)
    return pd.concat(df_list, axis="columns", join="outer")

## File Loading

In [6]:
df_list = list()
for raw_basename in raw_basenames:
    if raw_basename == "500mAh3.xlsx":
        continue
    filename = os.path.join(raw_dirname, raw_basename)
    display(HTML("<h1>{}</h1>".format(raw_basename)))
    xl_file = pd.ExcelFile(filename)
    print(xl_file.sheet_names)
    
    load_df = xl_file.parse("load", header=[0, 1])
    voltage_df = xl_file.parse("voltage", header=[0, 1])
    temperature_df = xl_file.parse("temperature", header=[0, 1])
    
#     display(load_df.head(1))
#     display(voltage_df.head(1))
#     display(temperature_df.head(1))
    
    new_load_df = process_load_df(load_df)
    new_voltage_df = process_voltage_df(voltage_df)
    new_temperature_df = process_temperature_df(temperature_df)
    
#     display(new_load_df.head(1))
#     display(new_voltage_df.head(1))
#     display(new_temperature_df.head(1))
    
    data_df = [new_load_df, new_voltage_df, new_temperature_df]
    data_df = pd.concat(data_df, axis="columns", join="outer")
    
    new_name = raw_basename.replace("mAh", " mAh ").replace(".xlsx", "")
    data_df.columns = pd.MultiIndex.from_tuples([
        (new_name,) + column
        for column in data_df.columns
    ])
    
#     display(data_df.head(1))

    df_list.append(data_df)
data_df = pd.concat(df_list, axis="columns", join="outer")

def column_compare(column):
    first,second,third = column
    first = (int(first.split()[0]), int(first.split()[-1]))
    second = int(second.split()[0])
    if "2000" in third:
        third = 0
    elif "Voltage" in third:
        third = 1
    else:
        third = 2
    return first,second,third

data_df = data_df[sorted(data_df.columns, key=column_compare)]

data_df.info(verbose=True)
with pd.option_context("display.max_rows", 10, "display.max_columns", None):
    display(data_df)
    display(data_df.describe())

['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']


['load', 'voltage', 'temperature']
<class 'pandas.core.frame.DataFrame'>
Float64Index: 30009 entries, 0.0 to 3011.4
Data columns (total 105 columns):
 #   Column                                       Dtype  
---  ------                                       -----  
 0   (500 mAh 1, 20 SOC, 2000 Pounds [Pounds])    float64
 1   (500 mAh 1, 20 SOC, Voltage [V])             float64
 2   (500 mAh 1, 20 SOC, Temperature [C])         float64
 3   (500 mAh 1, 40 SOC, 2000 Pounds [Pounds])    float64
 4   (500 mAh 1, 40 SOC, Voltage [V])             float64
 5   (500 mAh 1, 40 SOC, Temperature [C])         float64
 6   (500 mAh 1, 60 SOC, 2000 Pounds [Pounds])    float64
 7   (500 mAh 1, 60 SOC, Voltage [V])             float64
 8   (500 mAh 1, 60 SOC, Temperature [C])         float64
 9   (500 mAh 1, 80 SOC, 2000 Pounds [Pounds])    float64
 10  (500 mAh 1, 80 SOC, Voltage [V])             float64
 11  (500 mAh 1, 80 SOC, Temperature [C])         float64
 12  (500 mAh 1, 100 SOC, 2000 Pounds 

500 mAh 1                              \
                         20 SOC                               
           2000 Pounds [Pounds] Voltage [V] Temperature [C]   
Time [Sec]                                                    
0.0                         NaN         NaN          23.682   
0.1                    0.715723    3.745728             NaN   
0.2                         NaN         NaN          23.680   
0.3                    0.408984    3.745117             NaN   
0.4                    0.357861    3.745728          23.683   
...                         ...         ...             ...   
3010.6                      NaN         NaN             NaN   
3010.8                      NaN         NaN             NaN   
3011.0                      NaN         NaN             NaN   
3011.2                      NaN         NaN             NaN   
3011.4                      NaN         NaN             NaN   

                                                             \
                         40 SOC                               
           2000 Pounds [Pounds] Voltage [V] Temperature [C]   
Time [Sec]                                                    
0.0                         NaN         NaN          23.832   
0.1                         NaN         NaN             NaN   
0.2                         NaN         NaN          23.841   
0.3                    0.920215    3.781433             NaN   
0.4                    0.817969    3.781738          23.829   
...                         ...         ...             ...   
3010.6                      NaN         NaN             NaN   
3010.8                      NaN         NaN             NaN   
3011.0                      NaN         NaN             NaN   
3011.2                      NaN         NaN             NaN   
3011.4                      NaN         NaN             NaN   

                                                             \
                         60 SOC                               
           2000 Pounds [Pounds] Voltage [V] Temperature [C]   
Time [Sec]                                                    
0.0                         NaN         NaN          24.319   
0.1                    1.175830    3.872986             NaN   
0.2                         NaN         NaN          24.311   
0.3                    0.408984    3.873291             NaN   
0.4                    0.869092    3.873596          24.314   
...                         ...         ...             ...   
3010.6                      NaN         NaN             NaN   
3010.8                      NaN         NaN             NaN   
3011.0                      NaN         NaN             NaN   
3011.2                      NaN         NaN             NaN   
3011.4                      NaN         NaN             NaN   

                                                             \
                         80 SOC                               
           2000 Pounds [Pounds] Voltage [V] Temperature [C]   
Time [Sec]                                                    
0.0                         NaN         NaN          23.816   
0.1                    0.664600    3.998108             NaN   
0.2                         NaN         NaN          23.825   
0.3                    0.408984    3.997803             NaN   
0.4                    0.306738    3.997498          23.825   
...                         ...         ...             ...   
3010.6                      NaN         NaN             NaN   
3010.8                      NaN         NaN             NaN   
3011.0                      NaN         NaN             NaN   
3011.2                      NaN         NaN             NaN   
3011.4                      NaN         NaN             NaN   

                                                             \
                        100 SOC                               
           2000 Pounds [Pounds] Voltage [V] Temperature [C]   
Time [Sec]                                       

500 mAh 1                                                     \
                    20 SOC                                             40 SOC   
      2000 Pounds [Pounds]   Voltage [V] Temperature [C] 2000 Pounds [Pounds]   
count         11917.000000  11917.000000     7167.000000         12720.000000   
mean           -125.267985      0.394059       27.662398          -117.296458   
std              37.909750      1.108918        6.976766            39.503107   
min            -335.571680     -0.064392       23.606000          -345.080566   
25%            -129.954785      0.000610       24.059000          -121.417236   
50%            -124.229004      0.001831       24.283000          -114.106641   
75%            -122.030713      0.021667       26.271500          -111.959473   
max               1.635938      3.747864       48.749000             2.198291   

                                                                        \
                                                  60 SOC                 
        Voltage [V] Temperature [C] 2000 Pounds [Pounds]   Voltage [V]   
count  12720.000000     7629.000000         13660.000000  13660.000000   
mean       0.433031       28.652098          -118.967192      0.405137   
std        1.123398        8.973290            39.391807      1.066780   
min       -0.059509       23.577000          -358.065820     -0.057678   
25%        0.006409       23.990000          -123.308789      0.007019   
50%        0.010376       24.518000          -113.339795      0.012207   
75%        0.089416       28.254000          -110.630273      0.162048   
max        3.783569       63.111000             1.635938      3.876953   

                                                                          \
                                    80 SOC                                 
      Temperature [C] 2000 Pounds [Pounds]   Voltage [V] Temperature [C]   
count     8189.000000         13558.000000  13558.000000     8134.000000   
mean        35.512722          -105.082386      0.362067       39.524134   
std         18.611983            32.176086      1.063149       19.054216   
min         24.012000          -306.482666     -0.057068       23.303000   
25%         24.708000          -113.851025      0.005188       26.147250   
50%         26.137000          -101.019141      0.011597       28.877000   
75%         35.486000           -96.520313      0.115356       51.577750   
max         92.944000             0.971338      3.999634       95.381000   

                                                                    500 mAh 2  \
                   100 SOC                                             20 SOC   
      2000 Pounds [Pounds]   Voltage [V] Temperature [C] 2000 Pounds [Pounds]   
count         20621.000000  20621.000000    12341.000000          7974.000000   
mean           -104.556580      0.230287       31.738533           -97.893185   
std              39.005488      0.921235       10.676061            67.349226   
min            -354.538330     -0.058594       23.789000          -361.388818   
25%            -114.362256      0.002136       25.347000          -125.443176   
50%            -105.313477      0.003967       27.099000          -117.787500   
75%            -101.888232      0.016785       33.820000           -22.979810   
max               2.965137      4.133606       77.753000             4.601074   

                                                                      \
                                                 40 SOC                
       Voltage [V] Temperature [C] 2000 Pounds [Pounds]  Voltage [V]   
count  7974.000000     4772.000000          9061.000000  9061.000000   
mean      0.559564       31.155558           -75.694653     0.538796   
std       1.289850       12.188198            54.213870     1.190594   
min      -0.072021       23.631000          -342.013184    -0.039978   
25%       0.001831       24.337000           -95.395605     0.007019   
50%  

# Save Data

In [7]:
filename = os.path.join(results_dirname, results_basename)
print(filename)
data_df.to_csv(filename, compression="gzip")

../data/raw_mechanical_loading_data.csv.gz
